In [174]:
import warnings
warnings.simplefilter(action='ignore')
import os
import json
#import geopandas as gpd
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,  StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.utils.fixes import loguniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, accuracy_score, classification_report

# Configure Notebook
#for plots to be inline
%matplotlib inline 
#for auto_complete 
%config Completer.use_jedi = False 

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

In [2]:
!pip install folium

In [3]:
import folium

In [86]:
train_df = pd.read_csv('clean_weather_station.csv',index_col=[0])

In [87]:
train_df = train_df.dropna()
train_df.isnull().sum()
train_df = train_df.drop('index', axis=1)

In [88]:
train_df.head()

,time,bikes_available,docks_available,distance_to_nearest_transit_Miles,Latitude,Longitude,merge_time,max_temperature_f,mean_temperature_f,min_temperature_f,...,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code,percentage,50%
bike_station_name,,,,,,,,,,,,,,,,,,,,,
2nd at Folsom,2013-08-29 12:00:00,9.500000,9.500000,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,61.0,...,10.0,23.0,11.0,0.0,4.0,clear,286.0,94107,0.500000,1.0
2nd at Folsom,2013-08-29 13:00:00,9.264151,9.735849,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,61.0,...,10.0,23.0,11.0,0.0,4.0,clear,286.0,94107,0.512413,1.0
2nd at Folsom,2013-08-29 14:00:00,9.928571,9.071429,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,61.0,...,10.0,23.0,11.0,0.0,4.0,clear,286.0,94107,0.477444,0.0
2nd at Folsom,2013-08-29 15:00:00,10.055556,8.944444,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,61.0,...,10.0,23.0,11.0,0.0,4.0,clear,286.0,94107,0.470760,0.0
2nd at Folsom,2013-08-29 16:00:00,10.000000,9.000000,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,61.0,...,10.0,23.0,11.0,0.0,4.0,clear,286.0,94107,0.473684,0.0


In [89]:
train_df.describe()

,bikes_available,docks_available,distance_to_nearest_transit_Miles,Latitude,Longitude,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees,zip_code,percentage,50%
count,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,...,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1.204833e+06,1204833.0,1.204833e+06,1.204833e+06
mean,8.395349e+00,9.284578e+00,4.225844e+00,3.759259e+01,-1.222206e+02,6.843928e+01,6.046010e+01,5.201746e+01,5.254779e+01,4.904518e+01,...,9.542784e+00,7.875430e+00,1.891510e+01,8.206717e+00,3.332636e-02,3.854970e+00,2.548167e+02,94107.0,5.238836e-01,5.523604e-01
std,3.935228e+00,4.121336e+00,5.961433e+00,2.019536e-01,2.077293e-01,7.253263e+00,6.121853e+00,6.407280e+00,5.468569e+00,6.684713e+00,...,1.031418e+00,2.885090e+00,5.734225e+00,3.619720e+00,1.768981e-01,2.273887e+00,6.820783e+01,0.0,1.899110e-01,4.972510e-01
min,0.000000e+00,0.000000e+00,1.691848e-03,3.732973e+01,-1.224190e+02,4.700000e+01,4.100000e+01,3.200000e+01,2.700000e+01,1.700000e+01,...,4.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,94107.0,0.000000e+00,0.000000e+00
25%,6.000000e+00,6.333333e+00,3.348312e-02,3.738922e+01,-1.224008e+02,6.300000e+01,5.600000e+01,4.700000e+01,4.900000e+01,4.600000e+01,...,1.000000e+01,7.000000e+00,1.500000e+01,6.000000e+00,0.000000e+00,2.000000e+00,2.520000e+02,94107.0,4.000000e-01,0.000000e+00
50%,8.000000e+00,9.000000e+00,1.935685e-01,3.777106e+01,-1.223880e+02,6.800000e+01,6.000000e+01,5.200000e+01,5.300000e+01,4.900000e+01,...,1.000000e+01,1.000000e+01,2.000000e+01,8.000000e+00,0.000000e+00,4.000000e+00,2.750000e+02,94107.0,5.333333e-01,1.000000e+00
75%,1.056667e+01,1.198333e+01,8.202581e+00,3.778845e+01,-1.220767e+02,7.300000e+01,6.500000e+01,5.700000e+01,5.700000e+01,5.400000e+01,...,1.000000e+01,1.000000e+01,2.200000e+01,1.000000e+01,0.000000e+00,6.000000e+00,2.940000e+02,94107.0,6.611111e-01,1.000000e+00
max,2.700000e+01,2.700000e+01,1.817737e+01,3.780477e+01,-1.218773e+02,9.300000e+01,7.700000e+01,6.500000e+01,6.600000e+01,6.200000e+01,...,1.000000e+01,1.000000e+01,4.300000e+01,2.200000e+01,3.120000e+00,8.000000e+00,3.600000e+02,94107.0,1.000000e+00,1.000000e+00


In [90]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1204833 entries, 2nd at Folsom to Yerba Buena Center of the Arts (3rd @ Howard)
Data columns (total 31 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   time                               1204833 non-null  object 
 1   bikes_available                    1204833 non-null  float64
 2   docks_available                    1204833 non-null  float64
 3   distance_to_nearest_transit_Miles  1204833 non-null  float64
 4   Latitude                           1204833 non-null  float64
 5   Longitude                          1204833 non-null  float64
 6   merge_time                         1204833 non-null  object 
 7   max_temperature_f                  1204833 non-null  float64
 8   mean_temperature_f                 1204833 non-null  float64
 9   min_temperature_f                  1204833 non-null  float64
 10  max_dew_point_f                    1204833 no

In [91]:
train_df.columns

Index(['time', 'bikes_available', 'docks_available',
       'distance_to_nearest_transit_Miles', 'Latitude', 'Longitude',
       'merge_time', 'max_temperature_f', 'mean_temperature_f',
       'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f',
       'min_dew_point_f', 'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'precipitation_inches', 'cloud_cover', 'events',
       'wind_dir_degrees', 'zip_code', 'percentage', '50%'],
      dtype='object')

In [98]:
# Modify Events Column to be used in Select K Best
train_df['events'].replace({'clear': 0, 'fog': 1, 
                            'rain': 2, 'fog-rain': 3, 
                            'rain-thunderstorm': 4}, 
                            inplace=True)

# Create hour column so that a time can be incorporated into model
train_df['time']=pd.to_datetime(train_df['time'])
train_df['hour'] = train_df['time'].dt.hour

# Feature Engineering

In [168]:
# First we split the data into a train and test set
train, test = train_test_split(train_df, test_size = 0.3, random_state = 0, stratify=train_df['50%'])
val, test = train_test_split(test, test_size = 0.5, random_state = 0)

# We also define the target variable for training
y_train = train['50%']

print('Train {}%'.format(train.shape[0] / train_df.shape[0] * 100))
print('Test {}%'.format(test.shape[0] / train_df.shape[0] * 100))
print('Val {}%'.format(val.shape[0] / train_df.shape[0] * 100))

Train 69.99999170009453%
Test 15.000004149952732%
Val 15.000004149952732%


In [170]:
train.columns

Index(['time', 'bikes_available', 'docks_available',
       'distance_to_nearest_transit_Miles', 'Latitude', 'Longitude',
       'merge_time', 'max_temperature_f', 'mean_temperature_f',
       'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f',
       'min_dew_point_f', 'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'precipitation_inches', 'cloud_cover', 'events',
       'wind_dir_degrees', 'zip_code', 'percentage', '50%', 'hour'],
      dtype='object')

In [134]:
# Note columns that are irrelevant
columns_to_exclude = ['percentage', '50%', 'zip_code', 
                      'time', 'merge_time', 'bikes_available', 
                      'docks_available']

In [140]:
# Feature Selection
X_train_numerical = train.copy()[['max_visibility_miles', 'max_wind_Speed_mph', 
                                  'max_sea_level_pressure_inches', 'distance_to_nearest_transit_Miles', 
                                  'cloud_cover', 'hour', 'max_humidity', 'mean_wind_speed_mph', 
                                  'mean_temperature_f', 'mean_sea_level_pressure_inches', 
                                  'max_temperature_f', 'min_dew_point_f', 'wind_dir_degrees', 
                                  'min_visibility_miles', 'mean_dew_point_f', 'min_humidity', 
                                  'events', 'mean_visibility_miles', 'min_sea_level_pressure_inches', 
                                  'mean_humidity', 'max_dew_point_f', 'precipitation_inches', 
                                  'min_temperature_f']]

In [141]:
five_fold = KFold(n_splits=5)

X_new = SelectKBest(chi2, k=19).fit_transform(X_train_numerical, y_train)
X_new = pd.DataFrame(X_new)

# Iterature through cv-folds
for train_index, val_index in five_fold.split(X_new):
    
    # Select the 10 best features from X_train_numerical, y_train
    select = SelectKBest(chi2, k=19).fit(X_train_numerical.iloc[train_index, :],
                                       y_train.iloc[train_index])
    
    # Create reduced Feature set
    X_new = X_train_numerical.loc[:, select.get_support()]
    print("Selected features: {}".format(X_new.columns.tolist()))

Selected features: ['max_wind_Speed_mph', 'max_sea_level_pressure_inches', 'distance_to_nearest_transit_Miles', 'cloud_cover', 'hour', 'max_humidity', 'mean_wind_speed_mph', 'mean_temperature_f', 'max_temperature_f', 'min_dew_point_f', 'wind_dir_degrees', 'min_visibility_miles', 'mean_dew_point_f', 'min_humidity', 'events', 'mean_humidity', 'max_dew_point_f', 'precipitation_inches', 'min_temperature_f']
Selected features: ['max_wind_Speed_mph', 'max_sea_level_pressure_inches', 'distance_to_nearest_transit_Miles', 'cloud_cover', 'hour', 'max_humidity', 'mean_wind_speed_mph', 'mean_temperature_f', 'max_temperature_f', 'min_dew_point_f', 'wind_dir_degrees', 'min_visibility_miles', 'mean_dew_point_f', 'min_humidity', 'events', 'mean_humidity', 'max_dew_point_f', 'precipitation_inches', 'min_temperature_f']
Selected features: ['max_wind_Speed_mph', 'distance_to_nearest_transit_Miles', 'cloud_cover', 'hour', 'max_humidity', 'mean_wind_speed_mph', 'mean_temperature_f', 'max_temperature_f', 'm

In [158]:
features_total = ['max_wind_Speed_mph', 'max_sea_level_pressure_inches', 
                  'distance_to_nearest_transit_Miles', 'cloud_cover', 
                  'hour', 'max_humidity', 'mean_temperature_f',
                  'min_dew_point_f', 'wind_dir_degrees', 'min_visibility_miles', 
                  'events', 'max_dew_point_f', 'precipitation_inches', 'Latitude', 'Longitude']
features_10 = ['max_wind_Speed_mph', 'max_sea_level_pressure_inches', 
                  'distance_to_nearest_transit_Miles', 'cloud_cover', 
                  'hour', 'max_humidity', 'mean_temperature_f',
                  'min_dew_point_f', 'Latitude', 'Longitude']

X_train_total = train[features_total]
X_train_10 = train[features_10]
y_train = train['50%']

X_test_total = test[features_total]
X_test_10 = test[features_10]
y_test = test['50%']

In [105]:
model1 = LogisticRegression()

cv=KFold(n_splits=5)

param_grid = {'penalty': ('none', 'l1', 'l2'), 'fit_intercept':(True, False), 
             'C': (0.001, 0.01, 0.1, 1, 10, 100)}

search = GridSearchCV(model1, param_grid, scoring = 'f1', cv=cv, n_jobs=-1, verbose=True)
search.fit(X_train_total, y_train)

search.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


{'C': 10, 'fit_intercept': True, 'penalty': 'l2'}

In [113]:
model2 = RandomForestClassifier()

cv=KFold(n_splits=5)

param_grid = {'n_estimators': [10, 20, 100], 'max_features': ['auto','sqrt','log2'],
              'max_depth' : [None, 2, 4]}

search = GridSearchCV(model2, param_grid, scoring = 'f1', cv=cv, n_jobs=-1, verbose=True)
search.fit(X_train_total, y_train)

search.best_params_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


{'max_depth': None, 'max_features': 'auto', 'n_estimators': 20}

In [114]:
model3 = LogisticRegression()

cv=KFold(n_splits=5)

param_grid = {'penalty': ('none', 'l1', 'l2'), 'fit_intercept':(True, False), 
             'C': (0.001, 0.01, 0.1, 1, 10, 100)}

search = GridSearchCV(model3, param_grid, scoring = 'f1', cv=cv, n_jobs=-1, verbose=True)
search.fit(X_train_10, y_train)

search.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


{'C': 0.001, 'fit_intercept': False, 'penalty': 'l2'}

In [115]:
model4 = RandomForestClassifier()

cv=KFold(n_splits=5)

param_grid = {'n_estimators': [10, 20, 100, 200], 'max_features': ['auto','sqrt','log2'], 
              'max_depth' : [None, 2, 4]}

search = GridSearchCV(model4, param_grid, scoring = 'f1', cv=cv, n_jobs=-1, verbose=True)
search.fit(X_train_10, y_train)

search.best_params_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


{'max_depth': None, 'max_features': 'auto', 'n_estimators': 100}

In [159]:
# Use the optimal parameters for all 4 models on train data

model1 = LogisticRegression(C = 10, fit_intercept = True , penalty = 'l2')
model1.fit(X_train_total,y_train)
y_pred1 = model1.predict(X_train_total)

model2 = RandomForestClassifier(n_estimators = 20, max_features = 'auto', max_depth = None)
model2.fit(X_train_total,y_train)
y_pred2 = model2.predict(X_train_total)

model3 = LogisticRegression(C = 0.001, fit_intercept = False, penalty = 'l2')
model3.fit(X_train_10,y_train)
y_pred3 = model3.predict(X_train_10)

model4 = RandomForestClassifier(n_estimators= 100, max_features= 'auto', max_depth= None)
model4.fit(X_train_10,y_train)
y_pred4 = model4.predict(X_train_10)

In [160]:
# Now evaluate all of the models with the train data

f1_1 = f1_score(y_train, y_pred1)
print('The F1 Score for model 1 {}'.format(f1_1))

f1_2 = f1_score(y_train, y_pred2)
print('The F1 Score for model 2 {}'.format(f1_2))

f1_3 = f1_score(y_train, y_pred3)
print('The F1 Score for model 3 {}'.format(f1_3))

f1_4 = f1_score(y_train, y_pred4)
print('The F1 Score for model 4 {}'.format(f1_4))

The F1 Score for model 1 0.7092673124516509
The F1 Score for model 2 0.9988101911211048
The F1 Score for model 3 0.7078075242438303
The F1 Score for model 4 0.9999935601588495


In [161]:
# Use the optimal parameters for all 4 models on test data

model1 = LogisticRegression(C = 10, fit_intercept = True , penalty = 'l2')
model1.fit(X_test_total,y_test)
y_pred_1 = model1.predict(X_test_total)

model2 = RandomForestClassifier(n_estimators = 20, max_features = 'auto', max_depth = None)
model2.fit(X_test_total,y_test)
y_pred_2 = model2.predict(X_test_total)

model3 = LogisticRegression(C = 0.001, fit_intercept = False, penalty = 'l2')
model3.fit(X_test_10,y_test)
y_pred_3 = model3.predict(X_test_10)

model4 = RandomForestClassifier(n_estimators= 100, max_features= 'auto', max_depth= None)
model4.fit(X_test_10,y_test)
y_pred_4 = model4.predict(X_test_10)

In [162]:
# Now evaluate all of the models with the train data

f1_1 = f1_score(y_test, y_pred_1)
print('The F1 Score for model 1 {}'.format(f1_1))

f1_2 = f1_score(y_test, y_pred_2)
print('The F1 Score for model 2 {}'.format(f1_2))

f1_3 = f1_score(y_test, y_pred_3)
print('The F1 Score for model 3 {}'.format(f1_3))

f1_4 = f1_score(y_test, y_pred_4)
print('The F1 Score for model 4 {}'.format(f1_4))

The F1 Score for model 1 0.7072206378014
The F1 Score for model 2 0.9985094618956019
The F1 Score for model 3 0.7073801414817825
The F1 Score for model 4 0.9999949668567517


In [163]:
X_val_10 = val[features_10]
y_val = val['50%']

model4 = RandomForestClassifier(n_estimators= 100, max_features= 'auto', max_depth= None)
model4.fit(X_val_10,y_val)
y_pred__4 = model4.predict(X_val_10)

f1_4 = f1_score(y_val, y_pred__4)
print('The F1 Score for model 4 {}'.format(f1_4))

The F1 Score for model 4 1.0


In [164]:
cm = confusion_matrix(y_val, y_pred__4)

print('Confusion matrix\n', cm)
print('True Positives(TP) = ', cm[0,0])
print('True Negatives(TN) = ', cm[1,1])
print('False Positives(FP) = ', cm[0,1])
print('False Negatives(FN) = ', cm[1,0])
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]

Confusion matrix
 [[ 80415      0]
 [     0 100310]]
True Positives(TP) =  80415
True Negatives(TN) =  100310
False Positives(FP) =  0
False Negatives(FN) =  0


In [178]:
print(accuracy_score(y_val, y_pred__4))
print(f1_score(y_val, y_pred__4))
print(classification_report(y_val, y_pred__4))
print(confusion_matrix(y_val, y_pred__4))

1.0
1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     80415
         1.0       1.00      1.00      1.00    100310

    accuracy                           1.00    180725
   macro avg       1.00      1.00      1.00    180725
weighted avg       1.00      1.00      1.00    180725

[[ 80415      0]
 [     0 100310]]
